# Dispatch valuation

Routines for dispatch validation comprising
* Evaluation of imports and exports
* Evaluation of storage operation
* Evaluation of dispatch per energy carrier against historical one
* Evaluation of single dispatch situations
* Evaluation of energy not served and scarcity events

## Package imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from pommesevaluation.dispatch_validation import (
    load_entsoe_german_generation_data, plot_imports_and_exports, read_and_reshape_historical_im_ex
)

## Read in and filter for electrical bus results for Germany

In [ ]:
simulation_year = 2017
path_results = "./model_results/"
path_plots = "./plots/"
path_historical_production = "./data/production/"

model_file_name = f"dispatch_LP_start-{simulation_year}-01-01_364-days_simple_complete_production_UPDATE.csv"

In [ ]:
buses_el = pd.read_csv(path_results + model_file_name, index_col=0)
if simulation_year < 2022:
    historical_production = load_entsoe_german_generation_data(
        path=f"{path_historical_production}", year=simulation_year
    )

# Filter generation, exports and imports for Germany
bus_DE = buses_el[[col for col in buses_el.columns if "DE" in col]]

## Evaluate exports & imports
* Calculate overall exports and imports and net export
* Plot exports / imports by country
* Evaluate against historical exports and imports

### Calculate and plot overall imports and exports

In [ ]:
# Filter imports and exports and calculate overall and net imports & exports
im_ex_DE = bus_DE[[col for col in bus_DE.columns if "link_" in col]].copy()
export_links = [col for col in im_ex_DE if "('DE_link_" in col]
import_links = [col for col in im_ex_DE if "DE_bus_el')" in col]
im_ex_DE["overall_exports"] = im_ex_DE[export_links].sum(axis=1)
im_ex_DE["overall_imports"] = -im_ex_DE[import_links].sum(axis=1)
im_ex_DE["net_export"] = im_ex_DE["overall_exports"] + im_ex_DE["overall_imports"]
im_ex_DE.index = pd.to_datetime(im_ex_DE.index)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5)) 

_ = im_ex_DE[["overall_imports", "overall_exports"]].plot(ax=ax, color=["red", "blue"], alpha=0.3)
_ = im_ex_DE[["net_export"]].plot(ax=ax, color="black")
_ = ax.set_xlabel("Time")
_ = ax.set_ylabel("Energy in MWh")
_ = plt.legend(loc="upper right")
_ = plt.tight_layout()
_ = plt.savefig(f"{path_plots}overall_im_and_exports_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

### Evaluate imports and exports patterns for certain time frames

In [ ]:
im_ex_plot = im_ex_DE.copy()
im_ex_plot[import_links] *= -1
imports = {
    key: f"import_{key[3:5]}" 
    for key in import_links 
    if "DK" not in key and "SE" not in key
}
for key in import_links:
    if "DK" in key or "SE" in key:
        imports[key] = f"import_{key[3:6]}"

exports = {
    key: f"export_{key[11:13]}" 
    for key in export_links
    if "DK" not in key and "SE" not in key
}
for key in export_links:
    if "DK" in key or "SE" in key:
        exports[key] = f"export_{key[11:14]}"

im_ex_plot.rename(columns=imports, inplace=True)
im_ex_plot.rename(columns=exports, inplace=True)
im_ex_plot.drop(
    columns=[
        col for col in im_ex_plot.columns 
        if col not in imports.values() and col not in exports.values()
    ], 
    inplace=True
)

In [ ]:
country_colors = {
    "FR": "blue",
    "CH": "red",
    "BE": "black",
    "CZ": "gray",
    "DK1": "lightblue",
    "DK2": "turquoise",
    "PL": "dimgray",
    "NL": "orange",
    "NO": "green",
    "SE4": "yellow",
    "AT": "purple"
}

In [ ]:
colors_im = {
    f"import_{key}": val 
    for key, val in country_colors.items() 
    for col in im_ex_plot.columns if key in col
}
colors_ex = {
    f"export_{key}": val 
    for key, val in country_colors.items() 
    for col in im_ex_plot.columns if key in col
}
country_colors_im_ex = {**colors_im, **colors_ex}

In [ ]:
# Remove exchange with Austria before market splitting in 2018
if simulation_year <= 2018:
    im_ex_plot.drop(columns=["export_AT", "import_AT"], inplace=True)
    country_colors.pop("AT")
    country_colors_im_ex.pop("export_AT")
    country_colors_im_ex.pop("import_AT")

In [ ]:
plot_imports_and_exports(
    im_ex_plot, 
    country_colors_im_ex, 
    start=f"{simulation_year}-01-01 00:00", 
    end=f"{simulation_year}-01-07 23:00",
    save=True,
    path_plots=path_plots,
    file_name=f"imports_and_exports_{simulation_year}"
)

### Evaluate net exports / imports only
* Calculate sums for net exports

In [ ]:
for country in country_colors.keys():
    im_ex_plot[f"net_export_{country}"] = (
        im_ex_plot[f"export_{country}"]
        + im_ex_plot[f"import_{country}"]
    )
im_ex_plot.drop(
    columns=[
        col for col in im_ex_plot.columns
        if "net_export" not in col
    ],
    inplace=True
)

In [ ]:
for col in im_ex_plot.columns:
    im_ex_plot[f"{col}_pos"] = np.where(im_ex_plot[col] >= 0, im_ex_plot[col], 0)
    im_ex_plot[f"{col}_neg"] = np.where(im_ex_plot[col] < 0, im_ex_plot[col], 0)

In [ ]:
im_ex_plot = im_ex_plot.drop(
    columns=[
        col for col in im_ex_plot
        if "pos" not in col and "neg" not in col
    ]
)
im_ex_overall = im_ex_plot.copy()
im_ex_overall["overall_net_export"] = im_ex_overall.sum(axis=1)
im_ex_overall["overall_net_export_pos"] = np.where(
    im_ex_overall["overall_net_export"] >= 0,   
    im_ex_overall["overall_net_export"], 
    0
)
im_ex_overall["overall_net_export_neg"] = np.where(
    im_ex_overall["overall_net_export"] < 0, 
    im_ex_overall["overall_net_export"], 
    0
)

In [ ]:
country_colors_net_exports_pos = {
    f"net_export_{key}_pos": val 
    for key, val in country_colors.items() 
    for col in im_ex_plot.columns if key in col
}
country_colors_net_exports_neg = {
    f"net_export_{key}_neg": val 
    for key, val in country_colors.items() 
    for col in im_ex_plot.columns if key in col
} 
country_colors_net_exports = {
    **country_colors_net_exports_pos, **country_colors_net_exports_neg
}

In [ ]:
plot_imports_and_exports(
    im_ex_plot, 
    country_colors_net_exports, 
    start=f"{simulation_year}-01-01 00:00", 
    end=f"{simulation_year}-01-07 23:00",
    save=True,
    path_plots=path_plots,
    file_name=f"net_imports_and_exports_{simulation_year}"
)

### Compare modelled with historical exports and imports
* Read in and preprocess historical data
* Compare annual sums against each other

In [ ]:
if simulation_year < 2022:
    if not simulation_year == 2018:
        historical_im_ex = read_and_reshape_historical_im_ex(path_historical_production, simulation_year)
    else:
        historical_im_ex = pd.concat([
            read_and_reshape_historical_im_ex(path_historical_production, simulation_year),
            read_and_reshape_historical_im_ex(
                path_historical_production, 
                simulation_year, 
                file_name=f"Kommerzieller_Au_enhandel_{simulation_year}10010000_{simulation_year}12312359.xlsx"
            ),
        ])
else:
    historical_im_ex = read_and_reshape_historical_im_ex(path_historical_production, 2017)

In [ ]:
overall_net_exports = pd.DataFrame(index=historical_im_ex.columns)
overall_net_exports["model"] = im_ex_plot.sum()

if simulation_year < 2022:
    # Replace nan / string values before proceeding
    historical_im_ex.replace("----", 0, inplace=True)
    historical_im_ex = historical_im_ex.astype("float64")

    overall_net_exports["historical"] = historical_im_ex.sum()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
_ = overall_net_exports.plot(kind="bar", ax=ax)
_ = plt.axhline(y=0, color='gray', linestyle='-.', linewidth=.4)
_ = plt.title("Comparison of exports (pos) and imports (neg)")
_ = plt.tight_layout()
_ = plt.savefig(f"{path_plots}comparison_exports_imports_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

In [ ]:
# Calculate saldo of imports / exports
countries_modelled = ["DK1", "DK2", "NL", "CH", "CZ", "FR", "SE4", "PL"]
for country in countries_modelled:
    overall_net_exports.loc[country] = (
        overall_net_exports.loc[f"net_export_{country}_pos"] 
        + overall_net_exports.loc[f"net_export_{country}_neg"]
    )

overall_net_exports = overall_net_exports.loc[countries_modelled]

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
_ = overall_net_exports.plot(kind="bar", ax=ax)
_ = plt.axhline(y=0, color='gray', linestyle='-.', linewidth=.4)
_ = plt.title("Comparison of net exports (pos) and imports (neg)")
_ = plt.tight_layout()
_ = plt.savefig(f"{path_plots}comparison_exports_imports_saldo_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

## Evaluate storage operation

In [ ]:
storage_DE = bus_DE[[col for col in bus_DE.columns if "storage" in col]]
storage_DE_renamed = storage_DE.rename(columns={
    "(('DE_bus_el', 'DE_storage_el_PHS'), 'flow')": "storage_in",
    "(('DE_storage_el_PHS', 'DE_bus_el'), 'flow')": "storage_out"
})
storage_DE_renamed["storage_in"] *= -1
storage_DE_renamed["net_storage"] = storage_DE_renamed["storage_out"] + storage_DE_renamed["storage_in"]
storage_DE_renamed.index = pd.to_datetime(storage_DE_renamed.index)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))    


_ = ax.fill_between(
    storage_DE_renamed.index, 0, storage_DE_renamed.storage_in,
    step='post',
    facecolor='red',
    label="storage_in",
    alpha=1
)
_ = ax.fill_between(
    storage_DE_renamed.index, 0, storage_DE_renamed.storage_out,
    step='post',
    facecolor='blue',
    label="storage_out",
    alpha=1
)

_ = plt.axhline(y=0, color='gray', linestyle='-.', linewidth=.4)
_ = plt.title("Storage pattern")
_ = plt.tight_layout()
ax.set_xlabel("Time")
ax.set_ylabel("Energy in MWh")
_ = plt.legend()
_ = plt.tight_layout
_ = plt.savefig(f"{path_plots}storage_pattern_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

## Evaluate power generation

### Aggregate generation by fuel

In [ ]:
# Aggregate all generation units
power_generation_DE = bus_DE[[
    col for col in bus_DE.columns 
    if col not in im_ex_DE.columns
    and col not in storage_DE.columns
]]

# Drop demand, power prices and shortage
power_generation_DE = power_generation_DE.drop(
    columns=[
        "(('DE_bus_el', 'DE_sink_el_load'), 'flow')", 
        "(('DE_bus_el', 'None'), 'duals')",
        "(('DE_source_el_shortage', 'DE_bus_el'), 'flow')",
    ]
)

power_generation_DE["overall_generation"] = power_generation_DE.sum(axis=1)
power_generation_DE.index = pd.to_datetime(power_generation_DE.index)

In [ ]:
fuel_dict = {
    'ROR': 'Wasser',
    'biomass': 'Biomasse',
    'biomassEEG': 'Biomasse',
    'landfillgas': 'Deponiegas',
    'geothermal': 'Geothermie',
    'minegas': 'Grubengas',
    'larga': 'Klärgas',
    'windonshore': 'Windenergie an Land',
    'windoffshore': 'Windenergie auf See',
    'solarPV': 'Solare Strahlungsenergie',
    'uranium': 'Kernenergie',
    'lignite': 'Braunkohle',
    'hardcoal': 'Steinkohle',
    'waste': 'Abfall',
    'natgas': 'Erdgas',
    'otherfossil': 'Andere fossile',
    'mixedfuels': 'Mehrere fossile',
    'oil': 'Heizöl',
    'hydrogen': 'Wasserstoff',
}

In [ ]:
colors = {
    'solarPV': '#fcb001',
    'windonshore': '#82cafc',
    'windoffshore': '#0504aa',
    'uranium': '#e50000',
    'lignite': '#7f2b0a',
    'otherfossil': '#d8dcd6',
    'hardcoal': '#000000',
    'waste': '#c04e01',
    'mixedfuels': '#a57e52',
    'biomass': '#15b01a',
    'geothermal': '#ff474c',
    'otherres': '#06c2ac',
    'minegas': '#650021',
    'natgas': '#929591',
    'oil': '#aaa662',
    'ROR': '#c79fef',
    'storage_el_out' : 'darkblue',
    'hydrogen': '#6fa8dc',
}

In [ ]:
# Group outputs by energy carrier
energy_sources_dict = OrderedDict()

for fuel in fuel_dict.keys():
    energy_sources_dict[fuel] = [
        entry for entry in power_generation_DE.columns.values if fuel in entry
    ]   
    
# Store the aggregated production results per energy source
generation = pd.DataFrame()
for key, val in energy_sources_dict.items():
    generation[key] = power_generation_DE[val].sum(axis = 1)

# Aggregate
generation["biomass"] = generation["biomass"] + generation["biomassEEG"]
generation["otherres"] = (
    generation["landfillgas"]
    + generation["larga"]
)

generation.drop(
    columns=[
        col for col in generation.columns
        if col not in colors.keys()
    ],
    inplace=True
)

generation["storage_el_out"] = storage_DE_renamed["storage_out"]

### Visualize generation for certain time frames

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
_ = generation.iloc[4000:4369].plot(ax=ax, kind="area", color=colors)
_ = ax.set_xlabel("Time")
_ = ax.set_ylabel("Energy produced [MWh/h]")
_ = plt.legend(bbox_to_anchor=[1.02, 1.05])
_ = plt.tight_layout()
_ = plt.savefig(f"{path_plots}generation_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

### Compare summed generation againts historical one

In [ ]:
overall_generation = pd.DataFrame(index=generation.columns)
overall_generation["model"] = generation.sum()
if simulation_year < 2022:
    overall_generation["historical"] = historical_production.sum()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
_ = overall_generation.plot(kind="bar", ax=ax)
_ = plt.title("Comparison of production by fuel")
_ = plt.tight_layout()
_ = plt.savefig(f"{path_plots}production_by_fuel_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

In [ ]:
len(power_generation_DE.columns)

## Analyze dedicated supply situations
Analyze the supply situation for certain time steps / time frames

In [ ]:
start_time_step = f"{simulation_year}-01-01 00:00:00"
end_time_step = f"{simulation_year}-12-31 23:00:00"

In [ ]:
overall_situation = pd.concat(
    [
        storage_DE_renamed.loc[start_time_step:end_time_step, "storage_in"],
        generation.loc[start_time_step:end_time_step],
        im_ex_overall.loc[start_time_step:end_time_step, ["overall_net_export_pos", "overall_net_export_neg"]]
    ],
    axis = 1
)

In [ ]:
overall_situation.describe()

In [ ]:
start_time_step = f"{simulation_year}-04-30 00:00:00"
end_time_step = f"{simulation_year}-04-30 23:00:00"

In [ ]:
overall_situation_slice = overall_situation.loc[start_time_step: end_time_step].round(3)

In [ ]:
overall_situtation_colors = {
    **colors,
    "storage_in": "darkblue", 
    "overall_net_export_pos": "#ffefef",
    "overall_net_export_neg": "#ffefef"
}

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
_ = overall_situation_slice.plot(ax=ax, kind="area", color=overall_situtation_colors)
_ = ax.set_xlabel("Time")
_ = ax.set_ylabel("Energy [MWh/h]")
_ = plt.title(f"Dispatch situation between {start_time_step} and {end_time_step}")
_ = plt.legend(bbox_to_anchor=[1.02, 1.05])
_ = plt.tight_layout()
_ = plt.savefig(f"{path_plots}excess_situation_{simulation_year}.png", dpi=300)

plt.show()
plt.close()

In [ ]:
overall_situation_slice.loc[f"{simulation_year}-04-30 13:00"]

## Analyze scarcity and excess generation situations

### Calculate domestic net demand (after storage and exports / imports)

In [ ]:
demand_DE = bus_DE[["(('DE_bus_el', 'DE_sink_el_load'), 'flow')"]].rename(
    columns={
        "(('DE_bus_el', 'DE_sink_el_load'), 'flow')": "domestic_demand"
    }
)
demand_DE.index = pd.to_datetime(demand_DE.index)

In [ ]:
demand_DE.loc[f"{simulation_year}-04-30 13:00"]

### Contrast demand and generation for real scarcity situations (energy not served)
* Calculate generation after net storage and net exports and compare with domestic demand
* Identify the difference, i.e. shortages
* Identify maximum shortage value and time

In [ ]:
balance = power_generation_DE[["overall_generation"]].copy()
balance["net_export"] = im_ex_DE["net_export"]
balance["net_storage"] = storage_DE_renamed["net_storage"]
balance["gen_ex_stor"] = balance["overall_generation"] + balance["net_storage"] - balance["net_export"]
balance["demand"] = demand_DE["domestic_demand"]
balance["delta"] = balance["gen_ex_stor"] - balance["demand"]

In [ ]:
balance["delta"].loc[f"{simulation_year}-01-23 12:00":f"{simulation_year}-01-25"]

In [ ]:
# Identify scarcity situations
numeric_zero = 1e5
balance["delta"].loc[balance["delta"] < -numeric_zero]

In [ ]:
bus_DE["(('DE_source_el_shortage', 'DE_bus_el'), 'flow')"].max()

In [ ]:
bus_DE["(('DE_source_el_shortage', 'DE_bus_el'), 'flow')"].idxmax()

In [ ]:
# Excess situations occuring in AT; DE has no excess sink
excess_sinks = buses_el.loc[:,[col for col in buses_el.columns if "excess" in col]]
excess_sinks.loc[excess_sinks["(('AT_bus_el', 'AT_sink_el_excess'), 'flow')"] > 0.01]

### Evaluate scarcity situations where demand is met
Evaluate the dispatch of artificial scarcity units which are introduced to enable a market clearing in order to prevent energy being not served